# PC Pricing Calculator
Ignore all the code stuff. Scroll down to the good stuff!

Enter your allocations, editions, etc into the calculator.

Copy out your shiny new pricing!

Coming soon:

* Export to CSV/Excel/Sheets format?

* Add-ons

* General usability improvements

In [17]:
from __future__ import print_function
import math

import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from IPython.display import display, HTML

def pro_power_y(x):
    return 5.6271 * (x ** -0.296)
def ent_power_y(x):
    return 15.421 * (x ** -0.353)

def get_fee(edition, adj, discount, increase):
    adj = math.ceil((adj*(1+increase))/1000)*1000
    if edition=='Ent':
        fee = math.ceil((ent_power_y(adj)*adj)/100)*100*(1-discount)
    else:
        fee = math.ceil((pro_power_y(adj)*adj)/100)*100*(1-discount)
    return adj, fee

def tipping(current_fee, next_fee, overage, current_allocation):
    return round(((next_fee-current_fee)/(overage/100))+current_allocation)


def pandas_fmt(adf,mdf,currency):
    period='Annual'
    display(adf.style.format({
            'Effective Rate': "{:.2%}",
            'Overage Rate': "{:.2%}",
            'Tipping Point': f"{currency}"+"{0:,.0f}",
            f'{period} Allocation': f"{currency}"+"{0:,.0f}",
            f'{period} Fee': f"{currency}"+"{0:,.0f}"
        }))

    period='Monthly'
    display(mdf.style.format({
            'Effective Rate': "{:.2%}",
            'Overage Rate': "{:.2%}",
            'Tipping Point': f"{currency}"+"{0:,.0f}",
            f'{period} Allocation': f"{currency}"+"{0:,.0f}",
            f'{period} Fee': f"{currency}"+"{0:,.0f}"
        }))

def build_tiers(edition, payouts, increase, currency, discount, no_of_tiers, pc_pd):
    adj, fee = get_fee(edition, payouts, discount, increase)
    print('Base Tier:')
    print(f'    Payouts with {increase*100}% increase over historical: {currency}{adj:,}')
    print(f'    Fee: {currency}{round(fee):,}')
    print(f'    Discount Applied: {discount*100}%')
    print(f'    ER: {round(fee/adj*100,2)}%')
    
    if edition=='Ent':
        print(f'\nClosest tier: PC Enterprise Tier {min(enumerate(pc_pd["ent_payouts"]), key=lambda x: abs(x[1]-adj))[0]+1}')
    else:
        print(f'\nClosest tier: PC Pro Tier {min(enumerate(pc_pd["pro_payouts"]), key=lambda x: abs(x[1]-adj))[0]+1}')

    tiers = [
        [adj, fee, fee/adj, fee/adj*1.2]
    ]
    tier_names = []
    for i in range(no_of_tiers):
        if i == 0:
            tier_names.append('Base Tier')
        else:
            tier_names.append(f'Growth Tier {i}')
        adj, fee = get_fee(edition, adj, discount, increase)

        tiers.append([adj, fee, fee/adj, fee/adj*1.2])
    tier_names.append(f'Growth Tier {i+1}')

    for i in range(len(tiers)-1):
        tp = tipping(tiers[i][1],tiers[i+1][1],tiers[i][3],tiers[i][0])
        tiers[i].append(tp)

    tiers[-1].append(0)
    
    period='Annual'
    annual_df = pd.DataFrame(columns=[f'{period} Allocation', f'{period} Fee', 'Effective Rate', 'Overage Rate', 'Tipping Point'])
    for t in tiers:
        annual_df = annual_df.append({
            f'{period} Allocation': t[0],
            f'{period} Fee': t[1],
            'Effective Rate': t[2],
            'Overage Rate': t[3],
            'Tipping Point': t[4]
        }, ignore_index=True)
    annual_df.index=tier_names

    period='Monthly'
    monthly_df = pd.DataFrame(columns=[f'{period} Allocation', f'{period} Fee', 'Effective Rate', 'Overage Rate', 'Tipping Point'])
    for t in tiers:
        monthly_df = monthly_df.append({
            f'{period} Allocation': t[0]/12,
            f'{period} Fee': t[1]/12,
            'Effective Rate': t[2],
            'Overage Rate': t[3],
            'Tipping Point': t[4]/12
        }, ignore_index=True)
    monthly_df.index=tier_names

    pandas_fmt(annual_df, monthly_df,currency)
#    return annual_df, monthly_df, tiers


In [18]:
pc_pd=pd.read_csv('allocations.csv')

@interact
def show_tiers(payouts='500000', period=['Annual', 'Monthly'], edition=['Ent','Pro'], increase=(0,.5,.05), currency=['£', '€', '$'], discount=(.0, .75, .05), no_of_tiers=4):
    try:
        payouts = int(payouts)
        if period == 'Monthly':
            payouts = payouts *12
        if edition == 'Ent' and payouts < min(pc_pd['ent_payouts']):
            print(f'Sorry mate - that volume doesnt qualify for Enterprise pricing! Minimum is {currency}{round(min(pc_pd["ent_payouts"]))}')
        elif edition == 'Pro' and payouts < min(pc_pd['pro_payouts']):
            print(f'Sorry mate - that volume doesnt qualify for Pro pricing! Minimum is {currency}{round(min(pc_pd["pro_payouts"]))}')
        
        else:
            if discount >.5:
                print(f'Discount greater that 50% requires CRO approval - but start with your manager!')
            elif discount > .25:
                print(f'Discount greater that 25% requires MD/VP approval - but start with your manager!')
            elif discount > .1:
                print(f'Discount greater that 10% requires Sales Director or RVP approval.')
            else:
                print('')
            return build_tiers(edition, payouts, increase, currency, discount, no_of_tiers, pc_pd)
    except ValueError as e:
        print(f'Whoa! You sure that is a number there in the payout amount?\nSetting payouts to {currency}100,000\n', e)
        return


interactive(children=(Text(value='500000', description='payouts'), Dropdown(description='period', options=('An…